In [11]:
from flask import Flask,jsonify, request
from flask_cors import CORS, cross_origin

In [12]:
import pickle
import pandas as pd
import numpy as np

In [13]:
currency_rates={'AED': 0.27,
 'SAR': 0.27,
 'BHD': 2.65,
 'KWD': 3.25,
 'QAR': 0.27,
 'EUR': 1.0981,
 'CHF': 1.1191,
 'CZK': 0.047,
 'GBP': 1.2426,
 'RON': 0.2222,
 'USD': 1.0,
 'PLN': 0.2372,
 'HU1': 0.003,
 'SEK': 0.097,
 'CAD': 0.7484,
 'HKD': 0.1274,
 'AUD': 0.6714,
 'MYR': 0.2261,
 'SGD': 0.7514,
 'NZD': 0.6198}

In [14]:
with open('./model.pkl','rb') as f:
    model=pickle.load(f)

[01:22:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [15]:
with open('./scalar_feature.pkl','rb') as f:
    feature_scalar=pickle.load(f)

In [16]:
with open('./scalar_target.pkl','rb') as f:
    result_scalar=pickle.load(f)

In [17]:
encoders=["dc","div","oc","pot","cca"]

In [18]:
label_encoders={}
for i in encoders:
    with open('./encoder'+i+'.pkl','rb') as f:
        label_encoders[i]=pickle.load(f)

In [19]:
selected_features=['SALES_ORG', 'DISTRIBUTION_CHANNEL', 'DIVISION',
       'RELEASED_CREDIT_VALUE', 'PURCHASE_ORDER_TYPE', 'COMPANY_CODE',
       'ORDER_CREATION_TIME', 'CREDIT_CONTROL_AREA', 'SOLD_TO_PARTY',
       'ORDER_CURRENCY', 'CREDIT_STATUS', 'UNIQUE_CUST_ID', 'OCDATE',
       'OCMONTH', 'OCDAY', 'RDDATE', 'RDMONTH', 'RDDAY', 'DAYS_TO_PROCESS']

In [20]:
label_encoded_columns=['DISTRIBUTION_CHANNEL', 'DIVISION', 'PURCHASE_ORDER_TYPE','CREDIT_CONTROL_AREA', 'ORDER_CURRENCY']

In [21]:
def convert_currency(a,currency):
    return currency_rates[currency]*float(a)

In [22]:
df={}
def get_prediction(df):
    data=pd.DataFrame(df)
    ids=list(map(int,data['id']))
    currency=data['ORDER_CURRENCY']
    
    data['REQUESTED_DELIVERY_DATE']=pd.to_datetime(data['REQUESTED_DELIVERY_DATE'],format='%d-%m-%Y')
    data['ORDER_CREATION_DATE']=pd.to_datetime(data['ORDER_CREATION_DATE'],format='%d-%m-%Y')
    
    data['OCDATE']=data['ORDER_CREATION_DATE'].dt.day
    data['OCMONTH']=data['ORDER_CREATION_DATE'].dt.month
    data['OCDAY']=data['ORDER_CREATION_DATE'].dt.weekday

    data['RDDATE']=data['REQUESTED_DELIVERY_DATE'].dt.day
    data['RDMONTH']=data['REQUESTED_DELIVERY_DATE'].dt.month
    data['RDDAY']=data['REQUESTED_DELIVERY_DATE'].dt.weekday

    data['DAYS_TO_PROCESS']=(data['REQUESTED_DELIVERY_DATE']-data['ORDER_CREATION_DATE']).dt.days
    data=data[selected_features]
    
    data['CREDIT_STATUS'].replace('','74',inplace=True)
    data['CREDIT_STATUS']=data['CREDIT_STATUS'].astype('float64')
    data['PURCHASE_ORDER_TYPE'].fillna('NEWCAT',inplace=True)
    
    data['DISTRIBUTION_CHANNEL']=list(label_encoders['dc'].transform(data['DISTRIBUTION_CHANNEL']))
    data['DIVISION']=list(label_encoders['div'].transform(data['DIVISION']))
    data['PURCHASE_ORDER_TYPE']=list(label_encoders['pot'].transform(data['PURCHASE_ORDER_TYPE']))
    data['CREDIT_CONTROL_AREA']=list(label_encoders['cca'].transform(data['CREDIT_CONTROL_AREA']))
    data['ORDER_CURRENCY']=list(label_encoders['oc'].transform(data['ORDER_CURRENCY']))
    
    
    data['RELEASED_CREDIT_VALUE']=data['RELEASED_CREDIT_VALUE'].apply(lambda a:np.log(a+1))
    
    data['ORDER_CREATION_TIME']=data['ORDER_CREATION_TIME'].astype('int64')
    data=feature_scalar.transform(data)
    sacl=result_scalar.inverse_transform(model.predict(data).reshape(-1,1))
    
    
    
    resp=list(pd.DataFrame(sacl).apply(np.exp).values.flatten())
    amt_usd=list(map(convert_currency,resp,currency))
    
    place_holder=list(np.array([[*ids],[*resp],[*amt_usd]]).transpose())
    response=[]
    for i in place_holder:
        response.append(list(i))
        
    return response
    
    
    
    

In [23]:
# with open('./data.pkl','rb') as f:
#     print(get_prediction(pickle.load(f)))

In [24]:
pip install sklearn


In [25]:
app = Flask(__name__)
cors=CORS(app, origins='*')

@app.route('/predict', methods=["POST"])
def predict():
    resp=get_prediction(request.get_json())
    

    return jsonify({"data":(resp)})


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
